# Quick chat: send a Hello message via NexosAPI

This notebook demonstrates how to:
- Configure logging and environment variables for NexosAPI
- Prepare a minimal chat completion request
- Send the request and log the response

Prerequisites:
- NexosAPI configured with credentials available via a `.env` file
- Internet access
- A Jupyter environment that supports top-level `await`

## Setup: logging and environment

Enable informative logs and point NexosAPI to your `.env` file so it can load credentials and configuration automatically. Update the `NEXOSAI_INIT__DOTENV_PATH` to your local path.

In [1]:
import json
import logging
import os

# Set the logging level to desired verbosity
# This will help in debugging by showing detailed logs of the operations
# and interactions with the API.
from nexosapi.common.logging import setup_logging

LOGGING_LEVEL = logging.INFO
setup_logging(level=LOGGING_LEVEL)

# Set the environment variable to use dotenv to load environment variables

os.environ["NEXOSAI_INIT__DOTENV_PATH"] = (
    "/Users/kamilrybacki/Projects/nexos.api/examples/.env"  # Set the path to the .env file
)
os.environ["NEXOSAI_INIT__LOAD_DOTENV"] = "true"

## Prepare a simple chat completion request

Construct a minimal payload that sends a user message to the selected model. We preview the request payload with structured logging before sending it.

In [2]:
from nexosapi.api.endpoints import chat

# Define the parameters for the chat completion request
params = {
    "model": "8ef67c2a-5f50-4b19-acfe-5c2e404dbc95",
    "messages": [{"content": "Hello, how are you?", "name": "Kamil"}],
}
# Make a request to the chat completions endpoint with the specified parameters
chat.completions.request.prepare(params)

# Show the request before sending it
request_to_send = chat.completions.request.dump()
print(json.dumps(request_to_send, indent=2, ensure_ascii=False, sort_keys=True))

2025-08-17 14:19:14,146 - root - INFO - [SDK] Dependencies automatically wired.


{
  "frequency_penalty": 0.0,
  "messages": [
    {
      "content": "Hello, how are you?",
      "name": "Kamil",
      "role": "user"
    }
  ],
  "modalities": [
    "text"
  ],
  "model": "8ef67c2a-5f50-4b19-acfe-5c2e404dbc95",
  "n": 1,
  "presence_penalty": 0.0,
  "service_tier": "auto",
  "temperature": 1.0,
  "top_p": 1.0
}


## Send the request and log the response

Reload the last prepared request (if needed) and send it asynchronously. The response object is logged for inspection, including the model's message and token usage.

In [3]:
from nexosapi.domain.responses import ChatCompletionsResponse

# Send the "Hello" request to the chat completions endpoint
if chat.completions.request.pending is None:
    chat.completions.request.reload_last()
response: ChatCompletionsResponse = await chat.completions.request.send()

# Get the response from the chat completions endpoint
print(json.dumps(response.model_dump(), indent=2, ensure_ascii=False, sort_keys=True))

2025-08-17 14:19:14,635 - httpx - INFO - HTTP Request: POST https://api.nexos.ai/v1/chat/completions "HTTP/1.1 200 OK"


{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "annotations": [],
        "content": "Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?",
        "role": "assistant"
      }
    }
  ],
  "created": 1755433137,
  "id": "chatcmpl-C5Wddi2B2ZRepf89DFFvR0vfcGzGY",
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_560af6e559",
  "usage": {
    "completion_tokens": 30,
    "prompt_tokens": 16,
    "total_tokens": 46
  }
}
